In [1]:
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
import getpass

# Define Agents

In [2]:
API_Key= getpass.getpass("OPENAI API KEY: ")

In [3]:
llm_configs={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 0.9, "api_key": API_Key}]}

## Student

In [6]:
student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student, and you are looking for some idea for your topic to write your first research article.",
    llm_config=llm_configs
)

## Writer Assistant

In [7]:
introduction_agent = ConversableAgent(
    name="Introduction_Agent",
    system_message="You are a writer who very good at writing introduction. I will give you the topic, please write an introduction for a research article about that topic.",
    llm_config=llm_configs,
    human_input_mode="NEVER",
)

body_agent = ConversableAgent(
    name="Body_Agent",
    system_message="You are a writer who very good at writing body. I will give you the topic and the introduction, please write a body based on these parts.",
    llm_config=llm_configs,
    human_input_mode="NEVER",
)

conclusion_agent = ConversableAgent(
    name="Conclusion_Agent",
    system_message="You are a writer who very good at writing conclusion. I will give you the topic, the introduction and the body, please write a conclusion based on these parts.",
    llm_config=llm_configs,
    human_input_mode="NEVER",
)

# Sequential Chat

In [10]:
topic = "The future of Data Science."

# Step 1: Student sends the topic to Introduction_Agent
introduction_response = student_agent.initiate_chat(
    recipient=introduction_agent,
    message=f"This is my topic: {topic}.",
    max_turns=1,
    summary_method="last_msg",
)

# Step 2: Student sends topic + introduction to Body_Agent
body_response = student_agent.initiate_chat(
    recipient=body_agent,
    message=f"This is my topic and introduction: {introduction_response}.",
    max_turns=1,
    summary_method="last_msg",
)

# Step 3: Student sends topic + introduction + body to Conclusion_Agent
conclusion_response = student_agent.initiate_chat(
    recipient=conclusion_agent,
    message=f"This is my topic, introduction, and body: {body_response}.",
    max_turns=1,
    summary_method="last_msg",
)

# Step 4: Combine the results
chat_result = {
    "introduction": introduction_response,
    "body": body_response,
    "conclusion": conclusion_response,
}


Student_Agent (to Introduction_Agent):

This is my topic: The future of Data Science..

--------------------------------------------------------------------------------
Introduction_Agent (to Student_Agent):

Data science is a rapidly evolving field that has revolutionized the way businesses and organizations collect, analyze, and interpret data to make informed decisions. As technology continues to advance at an unprecedented pace, the future of data science holds endless possibilities and promises to reshape industries across the globe. In this research article, we will explore the emerging trends, challenges, and opportunities that lie ahead in the ever-growing field of data science. By gaining insight into the future of data science, we can better prepare ourselves for the transformative impact it will have on society and the way we interact with data.

--------------------------------------------------------------------------------
Student_Agent (to Body_Agent):

This is my topic 

# Group Chat

## Teacher Agent

In [11]:
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    llm_config=llm_configs,
    system_message="You are a music teacher. You will assign a topic to a group of students."
)

## Student agents

In [12]:
intro_student = ConversableAgent(
    name="Introduction_Student",
    system_message="You are a very good introduction writer. I will give you a topic, " 
    "please write an introduction for this topic.",
    llm_config=llm_configs,
    human_input_mode="NEVER",
)

body_student = ConversableAgent(
    name="Body_Student",
    system_message="You are a very good body writer. I will give you a topic and an introduction, " 
    "please write a body for an article of that topic.",
    llm_config=llm_configs,
    human_input_mode="NEVER",
)

conclusion_student = ConversableAgent(
    name="Conclusion_Student",
    system_message="You are a very good conclusion writer. I will give you a topic, an introduction and " 
    "abody, please write a conclusion for an article of that topic.",
    llm_config=llm_configs,
    human_input_mode="NEVER",
)

## Group Chat creating

In [13]:
from autogen import GroupChat

intro_student.description = "Introduction writer"
body_student.description = "Body writer"
conclusion_student.description = "Conclusion writer"

group_chat = GroupChat(
    agents=[intro_student, body_student, conclusion_student, teacher_agent],
    messages=[],
    max_round=4
)

In [14]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_configs
)

## Starting chat

In [15]:
chat_result = teacher_agent.initiate_chat(
    group_chat_manager,
    message="Today's topic is Modal Interchange.",
    summary_method="reflection_with_llm",
)

Teacher_Agent (to chat_manager):

Today's topic is Modal Interchange.

--------------------------------------------------------------------------------

Next speaker: Introduction_Student

Introduction_Student (to chat_manager):

Modal interchange is a fascinating musical concept that adds depth and complexity to compositions. By blending chords and scales from different modes within a single piece of music, composers can create rich harmonic textures and evoke a wide range of emotions. In this discussion, we will explore the theory behind modal interchange, its historical context, and how it is used in various genres of music to create compelling and unique sounds. Join me as we delve into the world of modal interchange and discover the endless creative possibilities it offers to musicians and listeners alike.

--------------------------------------------------------------------------------

Next speaker: Body_Student

Body_Student (to chat_manager):

Modal interchange, also known as 